# Model Training

In [1]:
import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
batchSize = 50
numClass = 10
epochs = 12

img_rows, img_cols = 28, 28

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [6]:
y_train = keras.utils.to_categorical(y_train, numClass)
y_test = keras.utils.to_categorical(y_test, numClass)

In [7]:
model = Sequential()
model.add(Conv2D(5, kernel_size=(5,5), \
                 activation='relu', input_shape=input_shape, \
                 padding='same', name='layer0'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(5, (5,5), activation='relu', padding='same', name='layer1'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(20, (5,5), activation='relu', padding='same', name='layer2'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(numClass, activation='softmax'))

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy, \
              optimizer=keras.optimizers.Adadelta(), \
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, \
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/12
1200/1200 [==============================] - 21s 18ms/step - loss: 2.3175 - accuracy: 0.1047 - val_loss: 2.3001 - val_accuracy: 0.1532
Epoch 2/12
1200/1200 [==============================] - 20s 17ms/step - loss: 2.3070 - accuracy: 0.1163 - val_loss: 2.2915 - val_accuracy: 0.2404
Epoch 3/12
1200/1200 [==============================] - 20s 17ms/step - loss: 2.2989 - accuracy: 0.1266 - val_loss: 2.2838 - val_accuracy: 0.2700
Epoch 4/12
1200/1200 [==============================] - 20s 17ms/step - loss: 2.2913 - accuracy: 0.1408 - val_loss: 2.2759 - val_accuracy: 0.2864
Epoch 5/12
1200/1200 [==============================] - 21s 18ms/step - loss: 2.2841 - accuracy: 0.1564 - val_loss: 2.2671 - val_accuracy: 0.3045
Epoch 6/12
1200/1200 [==============================] - 20s 17ms/step - loss: 2.2755 - accuracy: 0.1713 - val_loss: 2.2570 - val_accuracy: 0.3304
Epoch 7/12
1200/1200 [==============================] - 22s 18ms/step - loss: 2.2668 - accuracy: 0.1860 - val_loss: 2.2452 -

In [10]:
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  2.1465022563934326
Test accuracy:  0.5210000276565552


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Conv2D)              (None, 28, 28, 5)         130       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 5)         0         
_________________________________________________________________
layer1 (Conv2D)              (None, 14, 14, 5)         630       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 5)           0         
_________________________________________________________________
layer2 (Conv2D)              (None, 7, 7, 20)          2520      
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 20)          0         
_________________________________________________________________
flatten (Flatten)            (None, 980)               0

In [41]:
model.save('keras_model')

W0628 14:36:06.439682 140734809017792 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Filter Visualization

In [12]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def predict(idx, verbose=False):
    img = x_test[idx]
    label = y_test[idx]

    predict = model.predict_proba(np.expand_dims(img, axis=0))
    if verbose:
        print(f'Index: {idx}')
        cnt = 0
        for i in predict[0]:
            print('[{}] {:.2%}'.format(cnt, i))
            cnt += 1
        print('Predicted: ', predict[0].argmax())
        print('Label: ', label.argmax())
        plt.imshow(np.reshape(img, [28, 28]),interpolation='nearest', cmap='gray')
    return (predict[0].argmax(), label.argmax())

In [14]:
feature_extractor = keras.Model(inputs=model.inputs, \
                                outputs=[layer.output for layer in model.layers])

def get_hidden_layers(idx, _benchmark_=False):
    if _benchmark_:
        image = tf.convert_to_tensor(np.expand_dims(benchmark[idx], axis=0))
    else:
        image = tf.convert_to_tensor(np.expand_dims(x_test[idx], axis=0))
    features = feature_extractor(image)
    return features[0], features[2], features[4]

In [25]:
import math

def plotNNFilter(feature):
    feature = K.eval(feature)
    filters = feature.shape[3]  # number of filters
    #plt.figure(1, figsize=(20,20))
    n_columns = 5
    n_rows = math.ceil(filters / n_columns) + 1

    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(feature[0,:,:,i], interpolation='nearest', cmap='gray')

In [17]:
### Create benchmarks
# Take average of all the train images for each label.

In [16]:
from collections import defaultdict

label_class = defaultdict(list)
for i in range(y_train.shape[0]):
    label_class[y_train[i].argmax()].append(i)

In [17]:
# list of x_train data(28, 28, 1) for each label
lst = []
for j in range(10):
    lst.append([x_train[i] for i in label_class[j]])

In [18]:
# list of benchmarks for each label
benchmark = []
for i in range(10):
    tmp = []
    for j in range(len(lst[i])):
        tmp.append(lst[i][j])
    benchmark.append(np.mean(np.array(tmp), axis=0))

In [19]:
# dictionary of mal-predicted indices 
wrong = {}

for i in range(x_test.shape[0]):
    pred = predict(i)
    if pred[0] != pred[1]:
        wrong[i] = (pred[0], pred[1])

W0628 14:15:20.282444 140734809017792 deprecation.py:323] From <ipython-input-13-283aa33df260>:5: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.


In [43]:
#choice = []
choice = open('dropdown.txt', 'w')
for k, v in wrong.items():
    string = '%d) predicted: %d, label: %d' %(k, v[0], v[1])
    #choice.append(string)
    choice.write(string + '\n')

## ----------------------------------
# visit mnist_voila.ipynb

In [39]:
# compare and contrast by tensor subtraction
def plotCompare(feature1, feature2):
    if feature1.shape != feature2.shape:
        print("Shapes don't match")
        return

    feature1 = K.eval(feature1)
    feature2 = K.eval(feature2)
    filters = feature1.shape[3]  # number of filters
    #plt.figure(1, figsize=(20,20))
    n_columns = 5
    n_rows = math.ceil(filters / n_columns) + 1

    for i in range(filters):
        gap = tf.math.abs(tf.math.subtract(feature1[0,:,:,i], feature2[0,:,:,i]))
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(gap, interpolation='nearest', cmap='gray')

In [ ]:
### Interactive Widget

In [21]:
import ipywidgets as widgets
from ipywidgets import interact_manual
import seaborn as sns

In [22]:
choice = []
for k, v in wrong.items():
    string = '%d) predicted: %d, label: %d' %(k, v[0], v[1])
    choice.append(string)

In [40]:
@interact_manual
def filterViz(Image = choice, Layer = [0, 1, 2]):
    i = Image.split(')')
    idx = int(i[0])
    i = Image.split(',')
    predicted = int(i[0][-1])
    answer = int(i[1][-1])
    
    hiddens = get_hidden_layers(idx)    
    hiddens_p = get_hidden_layers(predicted, True)
    hiddens_a = get_hidden_layers(answer, True)
    
    %matplotlib inline
    sns.set_style('darkgrid')
    
    fig1 = plt.figure(1)
    fig1.suptitle(f'Predicted to be {predicted}; Answer is {answer}', fontsize=20)
    predict(idx, True)
    
    fig2 = plt.figure(2, figsize=(50,20))
    fig2.suptitle(f'Hidden layer{Layer} of the image', fontsize=50)
    plotNNFilter(hiddens[Layer])

    fig3 = plt.figure(3, figsize=(50,20))
    fig3.suptitle(f'Hidden layer{Layer} of benchmark {predicted}', fontsize=50)
    plotNNFilter(hiddens_p[Layer])
    
    fig4 = plt.figure(4, figsize=(50,20))
    fig4.suptitle(f'Hidden layer{Layer} of benchmark {answer}', fontsize=50)
    plotNNFilter(hiddens_a[Layer])
    
    fig5 = plt.figure(5, figsize=(50,20))
    fig5.suptitle(f'Compare n Contrast with benchmark {predicted}', fontsize=50)
    plotCompare(hiddens_p[Layer], hiddens[Layer])

    fig6 = plt.figure(6, figsize=(50,20))
    fig6.suptitle(f'Compare n Contrast with benchmark {answer}', fontsize=50)
    plotCompare(hiddens_a[Layer], hiddens[Layer])
    
    plt.show()

interactive(children=(Dropdown(description='Image', options=('1) predicted: 3, label: 2', '4) predicted: 9, la…